In [1]:
import pretty_midi
import symusic
import mido
import numpy as np
import joblib
import glob
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns
# sns.set_style('white')
# sns.set_context('notebook', font_scale=1.5)
import matplotlib.gridspec
import collections
import os
from pathlib import Path
# plotting.py contains utility functions for making nice histogram plots
import util.plotting as plotting
from util.play_midi import play_midi
from statistics import mean, stdev, variance
import random
import miditok.
import pandas as pd
from tqdm import tqdm 

from huggingface_hub import notebook_login
import ipywidgets

In [3]:
if "lofi_radio" in os.getcwd().split(os.sep)[-1].lower():
    os.chdir("..")

# if os.chdir("..")
print(os.getcwd())

midi_files = [str(file) for file in Path("pretraining_dataset/").glob("**/*.mid")]
print(len(midi_files))

c:\Users\joeac\Documents\Uni\Y3\Project
138174


In [4]:
tokenizer = miditok.REMI.from_pretrained("theglassofwater/remi_12500")

tokenizer.json:   0%|          | 0.00/675k [00:00<?, ?B/s]

c:\Users\joeac\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joeac\.cache\huggingface\hub\models--theglassofwater--remi_12500. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\joeac\AppData\Local\Programs\Python\Python311\Lib\site-packages\miditok\mi

In [7]:
seq_len = 100_000
full_dataset = miditok.pytorch_data.DatasetMIDI(
    files_paths=midi_files,
    max_seq_len=seq_len+1,  # max_seq_len = start + seq_len + end
    tokenizer=tokenizer,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

In [9]:
def tokenize_file(file):
    try:
        tokenized_file = file["input_ids"]
        return tokenized_file
    except:
        return None

tokenized_dataset = joblib.Parallel(n_jobs=6, verbose=1)(
    joblib.delayed(tokenize_file)(midi_file)
    for midi_file in full_dataset)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    9.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   29.2s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   58.5s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  1.7min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  2.6min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  3.7min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  5.1min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  6.7min
[Parallel(n_jobs=6)]: Done 4038 tasks      | elapsed:  8.5min
[Parallel(n_jobs=6)]: Done 4988 tasks      | elapsed: 10.6min
[Parallel(n_jobs=6)]: Done 6038 tasks      | elapsed: 12.9min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed: 15.4min
[Parallel(n_jobs=6)]: Done 8438 tasks      | elapsed: 18.4min
[Parallel(n_jobs=6)]: Done 9788 tasks      | elapsed: 21.4min
[Parallel(n_jobs=6)]: Done 11238 tasks      | elapsed: 24.7mi

In [ ]:
hf_csv = pd.Dataframe({"input_id":tokenized_dataset})
hf_csv.to_csv("tokenized_dataset.csv")

In [ ]:
from datasets import load_dataset

hf_dataset = load_dataset("pretraining_tokenized_dataset", data_dir="Users/joeac/Documents/Uni/Y3/Project/tokenized_dataset.csv"))
hf_dataset.push_to_hub("theglassofwater/pretraining_tokenized_dataset")